In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import Tensorboad
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import tensorflow as tf
import numpy as np


Using TensorFlow backend.


# 数据预处理

In [2]:
batch_size = 128
num_classes = 10
epochs = 1
img_rows, img_cols = 28, 28

tensorboad = Tensorboard(log_dir='log')
checkpoint = ModelCheckpoint(filepath='file_name',monitor='val_acc',mode='auto' ,save_best_only='True')

callback_lists=[tensorboard,checkpoint]
 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
 
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# keras模型搭建

In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

# 训练模型

In [4]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
 
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
         callbacks=callback_lists)


Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 1449s 24ms/step - loss: 0.2677 - acc: 0.9169 - val_loss: 0.0610 - val_acc: 0.9813


# 模型评估

In [5]:
loss,acc = model.evaluate(x_test, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 9s 875us/step
Restored model, accuracy: 98.13%


# Keras权重保存为h5

In [12]:
from keras.models import load_model
# 保存权重
model.save_weights('my_checkpoint/mnist.h5')

# kera模型保存为pb文件

In [19]:
sess = K.get_session()
 
frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    sess,
    sess.graph_def,
    output_node_names=["dense_2/Softmax"])
 
# 保存图为pb文件
# with open('model.pb', 'wb') as f:
#     f.write(frozen_graph_def.SerializeToString())
 
tf.train.write_graph(frozen_graph_def, 'model', 'mnist_model.pb', as_text=False)

ERROR! Session/line number was not unique in database. History logging moved to new session 360
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


'model\\mnist_model.pb'